In [2]:
from image import *

In [3]:
# RGB image
preprocess = {"nothing": lambda x: x, "contrast": increase_contrast, "normalize": normalize, "blackhat": blackhat}
hairs = {"nothing": lambda x: x, "median": median, "low_pass_filter": low_pass_filter_rgb, "dilate": dilate}
segments = {"threshold": threshold, "watershed": watershed, "mser": mser, "k_means": k_means}

img_folder = "images/original"
seg_folder = "images/segmented"

results = {}

for pre in preprocess:
    for hair in hairs:
        for seg in segments:

            name = "{} - {} - {}".format(pre, hair, seg)
            results[name] = {}
            avg_TPR = 0
            avg_FPR = 0
            avg_TP = 0
            avg_TN = 0
            avg_FP = 0
            avg_FN = 0
            cont = 0

            for img_name in sorted(os.listdir(img_folder)):
                cont += 1
                aux_index = img_name.index("i")
                n = img_name[0:aux_index]
                # Real image
                img_path = img_folder + "/" + img_name
                img = load_image_rgb(img_path)

                # Segmented by function
                func_seg_img = segments[seg](hairs[hair](preprocess[pre](img)))

                # Improve the segmentation by applying morphology operations
                opening = cv2.morphologyEx(func_seg_img, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
                final = cv2.morphologyEx(opening, cv2.MORPH_DILATE, np.ones((5,5),np.uint8),iterations=3)
                final = cv2.medianBlur(final,9)
                final = cv2.morphologyEx(final, cv2.MORPH_DILATE, np.ones((7,7),np.uint8),iterations=3)
                final = cv2.medianBlur(final,11)
                func_seg_img = cv2.morphologyEx(final, cv2.MORPH_CLOSE, np.ones((7,7),np.uint8))

                # Real segmentation
                real_seg_path = seg_folder + "/" + n + "seg.png"
                real_seg_img = load_image(real_seg_path)

                # Compare
                TPR, FPR, TP, TN, FP, FN = compare_seg(real_seg_img, func_seg_img)
                this_res = {
                    "TP": TP,
                    "FP": FP,
                    "TN": TN,
                    "FN": FN,
                    "TPR": TPR,
                    "FPR": FPR,
                }
                results[name][n] = this_res
                avg_TPR += TPR
                avg_FPR += FPR
                avg_TP += TP
                avg_TN += TN
                avg_FP += FP
                avg_FN += FN

            results[name]["average_TPR"] = avg_TPR/cont
            results[name]["average_FPR"] = avg_FPR/cont
            results[name]["average_TP"] = avg_TP/cont
            results[name]["average_TN"] = avg_TN/cont
            results[name]["average_FP"] = avg_FP/cont
            results[name]["average_FN"] = avg_FN/cont
            
            # print(name)
            # print("Avg TPR:", results[name]["average_TPR"])
            # print("Avg FPR:", results[name]["average_FPR"])
            # print()       



In [17]:
import json
# Random metric 
def metric2(TPR,FPR):
    return (1+(1-TPR))**2 + (1+FPR)**2 - 2

def metric1(TPR,FPR):
    return 1-TPR+FPR

# Save results
# with open('results/all_results.txt', 'w') as outfile:
#    json.dump(results, outfile)

name = "contrast - median - k_means"
avg_TPR = results[name]["average_TPR"]
avg_FPR = results[name]["average_FPR"]
simple_dif = metric1(avg_TPR,avg_FPR)
squared_dif = metric2(avg_TPR,avg_FPR)
for i in range(1,51):
    n = str(i)
    TPR = results[name][n]["TPR"]
    if TPR < 0.8:
        print("Low TPR:",n, "({})".format(TPR))

for i in range(1,51):
    n = str(i)
    FPR = results[name][n]["FPR"]
    if FPR > 0.2:
        print("High FPR:",n, "({})".format(FPR))



Low TPR: 2 (0.6862354833263731)
Low TPR: 8 (0.754207263064659)
Low TPR: 9 (0.7315679442508711)
Low TPR: 33 (0.5641339028406752)
Low TPR: 40 (0.6891365494843268)
High FPR: 13 (0.23914062167189695)
High FPR: 15 (0.22346267872966558)
High FPR: 18 (0.46405990663180624)
High FPR: 24 (0.5795129730724407)
High FPR: 26 (0.4325814990388196)
High FPR: 27 (0.2086287398596656)
High FPR: 28 (0.28061644391380447)
High FPR: 31 (0.34439182864379714)
High FPR: 32 (0.44229172432881264)
High FPR: 33 (0.46835732711991346)
High FPR: 35 (0.32011625690870976)
High FPR: 40 (0.6264988621557196)
